# Project Code

### Katie and Kristen's Code for ProjectCSGirls 2022

In [1]:
#Import libraries for use throughout our code

from IPython.display import clear_output, display, update_display #IPython is the computing shell for our programming!
import time #For general use
import smbus #Temperature

#Temperature and humidity
from bme280 import BME280
bme280 = BME280() 

import busio #Air quality sensor, buzzer
import board #Air quality sensor, buzzer, and display
from board import * #Air quality sensor, buzzer

#Air quality sensor
i2c = board.I2C() 

#Google Sheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime

#Adafruit
import os
import adafruit_ccs811
from Adafruit_IO import Client, Feed, RequestError

#Loading the .env file for later customization
from dotenv import load_dotenv
load_dotenv()  

#Screen display
import digitalio
from PIL import Image, ImageDraw, ImageFont
import adafruit_ssd1306

from adafruit_seesaw import seesaw, pwmout #Buzzer

Google Sheets Initial Code

In [2]:
sheet_name = "ProjectCSGirls"   #Define the name of our Google Sheet

file = os.environ.get("JSON") #Create a variable that takes the name of our secret.json file from the .env file

#use credentials to create a client that interacts with the Google Drive API
scopes = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file", 
          "https://www.googleapis.com/auth/spreadsheets"]
creds = ServiceAccountCredentials.from_json_keyfile_name(file, scopes)
client = gspread.authorize(creds)

#Find the workbook by name and open the first sheet
sheet = client.open(sheet_name).sheet1

Testing each measurement individually

In [3]:
#temperature

def readTemp():
    #By default the address of LM75A is set to 0x48
    address = 0x48

    #Read I2C data and calculate temperature
    bus = smbus.SMBus(1)
    raw = bus.read_word_data(address, 0) & 0xFFFF
    raw = ((raw << 8) & 0xFF00) + (raw >> 8)
    temperature = (raw / 32.0) / 8.0
    return temperature

In [4]:
#humidity
h = bme280.get_humidity()
print (h)

88.2681598298333


In [5]:
#CO2 and TVOC
ccs = adafruit_ccs811.CCS811(i2c)
print("CO2: ", ccs.eco2, "TVOC:", ccs.tvoc)

CO2:  None TVOC: None


Adafruit Connection

In [6]:
ADAFRUIT_IO_USERNAME = os.environ.get("ADAFRUIT_IO_USERNAME") #Get our Adafruit usernames from the .env file
ADAFRUIT_IO_KEY = os.environ.get("ADAFRUIT_IO_KEY") #Get our Adafruit passwords from the .env file

aio = Client(ADAFRUIT_IO_USERNAME, ADAFRUIT_IO_KEY)

Location

In [7]:
#Create more lat/lon variables
lat = os.environ.get("LAT")
lon = os.environ.get("LON")

#Create metadata variable
metadata = {"lat": lat,
            "lon": lon,
            "ele": 0,
            "created_at":None}

Reading Adafruit feeds

In [8]:
exterFeed = aio.feeds("external")
humidFeed = aio.feeds("humidity")
co2Feed = aio.feeds("co2")
tvocFeed = aio.feeds("tvoc")

Buzzer Initial Code

In [9]:
myI2C = busio.I2C(SCL, SDA)

ss = seesaw.Seesaw(myI2C)
pwm = pwmout.PWMOut(ss,5)

In [ ]:
Screen Initial Code 

In [16]:
#You need to install the driover using 
# sudo pip3 install adafruit-circuitpython-ssd1306

#Define the Reset Pin
oled_reset = digitalio.DigitalInOut(board.D4)

#Set the width and height of the screen
WIDTH = 128
HEIGHT = 32 

#Use for the i2c
i2c = board.I2C()
oled = adafruit_ssd1306.SSD1306_I2C(WIDTH, HEIGHT, i2c, addr=0x3C, reset=oled_reset)

#Clear the display
oled.fill(0)
oled.show()

#Make sure to create image with mode '1' for 1-bit color
image = Image.new("1", (oled.width, oled.height))

# Get drawing object to draw on image
draw = ImageDraw.Draw(image)

#Write some text on the screen
text = "Katie & Kristen"
font = ImageFont.truetype("arial.ttf", 9)
(font_width, font_height) = font.getsize(text)
draw.text( (oled.width // 2 - font_width // 2, -1), text, font=font, fill=255,)

text = "ProjectCSGirls"
font = ImageFont.truetype("arial.ttf", 9)
(font_width, font_height) = font.getsize(text)
draw.text( (oled.width // 2 - font_width // 2, 10), text, font=font, fill=255,)
 
text = "2022 <3"
font = ImageFont.truetype("arial.ttf", 9)
(font_width, font_height) = font.getsize(text)
draw.text( (oled.width // 2 - font_width // 2, 20), text, font=font, fill=255,)

#Display the text
oled.image(image)
oled.show()

Main Loop 

In [19]:
while True:
    #Define letter variables
    t = readTemp()
    e = (t * 9/5) + 32 
    h = bme280.get_humidity()
    co2 = ccs.eco2
    tvoc = ccs.tvoc
    locate = os.environ.get("LOCATION") #Create a variable that takes our location from the .env file
    
    #Send data to Adafruit feeds
    aio.send_data(exterFeed.key, e, metadata)
    aio.send_data(humidFeed.key, h, metadata)
    aio.send_data(co2Feed.key, co2, metadata)
    aio.send_data(tvocFeed.key, tvoc, metadata)
    
    #Send and print data on the Google Sheet
    now=datetime.now()
    now_time = now.strftime("%H:%M:%S")
    now_day = now.strftime("%m/%d/%Y")
    row=[now_day, now_time, e, h, co2, tvoc, locate]
    sheet.append_row(row)

    #Print data in Jupyter Notebooks (below)
    print(e, ":°Fahrenheit|")
    print(h, ":% Humidity|")
    print(co2, ": PPM (CO2)|")
    print(tvoc, ":PPB (TVOC)|")
    print(locate, ":Location of reading|")
    print()
    
    #Clear the LCD screen
    oled.fill(0)
    oled.show()
    
    #Tells the buzzer and screen what to do *TESTING*
    if ((e >= 60) and (co2 >= 402)):
      #Buzzer
      pwm.duty_cycle=0x80 #Makes a noise for 5 seconds at a single frequency
      time.sleep(1)
      pwm.duty_cycle=0x00 #Pause (silent) for 5 seconds
      time.sleep(2)
      pwm.duty_cycle=0x90 #Makes a noise for 5 seconds at a slightly higher but still linear frequency
      time.sleep(1)
      pwm.duty_cycle=0x00 #Pause (silent) for 5 seconds
      time.sleep(2)
        
      ##Screen display
      image = Image.new("1", (oled.width, oled.height))
        
      draw = ImageDraw.Draw(image)
    
      text = "EnviroReader" #top line of text
      font = ImageFont.truetype("arial.ttf", 9)
      (font_width, font_height) = font.getsize(text)
      draw.text( (oled.width // 2 - font_width // 2, -1), text, font=font, fill=255,)

      text = "ALERT!" #middle line of text
      font = ImageFont.truetype("arial.ttf", 9)
      (font_width, font_height) = font.getsize(text)
      draw.text( (oled.width // 2 - font_width // 2, 10), text, font=font, fill=255,)

      text = "Your CO2 is too high" #bottom line of text
      font = ImageFont.truetype("arial.ttf", 9)
      (font_width, font_height) = font.getsize(text)
      draw.text( (oled.width // 2 - font_width // 2, 20), text, font=font, fill=255,)

      oled.image(image) #show the text
      oled.show()
    else:
      ##Screen display
      text = "Katie & Kristen" #top line of text
      font = ImageFont.truetype("arial.ttf", 9)
      (font_width, font_height) = font.getsize(text)
      draw.text( (oled.width // 2 - font_width // 2, -1), text, font=font, fill=255,)

      text = "ProjectCSGirls" #middle line of text
      font = ImageFont.truetype("arial.ttf", 9)
      (font_width, font_height) = font.getsize(text)
      draw.text( (oled.width // 2 - font_width // 2, 10), text, font=font, fill=255,)

      text = "2022 <3" #bottom line of text
      font = ImageFont.truetype("arial.ttf", 9)
      (font_width, font_height) = font.getsize(text)
      draw.text( (oled.width // 2 - font_width // 2, 20), text, font=font, fill=255,)

      oled.image(image) #show the text
      oled.show()

    #Wait a certain amount of time before taking the next reading
    time.sleep(24)

65.975 :°Fahrenheit|
29.293655360838983 :% Humidity|
520 : PPM (CO2)|
18 :PPB (TVOC)|
Delaware :Location of reading|

65.975 :°Fahrenheit|
28.72759469867241 :% Humidity|
452 : PPM (CO2)|
7 :PPB (TVOC)|
Delaware :Location of reading|

65.975 :°Fahrenheit|
28.643349902796277 :% Humidity|
443 : PPM (CO2)|
6 :PPB (TVOC)|
Delaware :Location of reading|

66.2 :°Fahrenheit|
28.562202870583402 :% Humidity|
423 : PPM (CO2)|
3 :PPB (TVOC)|
Delaware :Location of reading|



KeyboardInterrupt: 

Screen Code - Basic display

In [ ]:
#Run this cell to stop the buzzer!
pwm.duty_cycle=0x00

In [14]:
#Clear the LCD screen
oled.fill(0)
oled.show()